In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os


In [ ]:
valid_dir = "/content/drive/MyDrive/proj/valid_0"

In [ ]:
val_dataset = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    image_size=(224, 224),
    batch_size=32,
    shuffle=True,
    seed=1
)


Found 8 files belonging to 9 classes.


In [ ]:
class_names = ["drop", "flow", "pool", "pattern", "swipe", "wiping","high","low","medium"]  # Update based on your dataset


In [ ]:
import os
from PIL import Image, ImageFile

# Increase max image pixel limit to prevent DecompressionBombError
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True  # Allows loading truncated/corrupt images

def remove_or_resize_large_images(directory, max_size=(1024, 1024)):
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            for filename in os.listdir(folder_path):
                file_path = os.path.join(folder_path, filename)
                try:
                    with Image.open(file_path) as img:
                        img.load()  # Load image to catch potential errors

                        # Convert to RGB (fixes transparency warning)
                        if img.mode in ("P", "RGBA", "LA"):
                            img = img.convert("RGB")

                        # Resize if too large
                        if img.size[0] * img.size[1] > 178956970:
                            print(f"⚠️ Resizing large image: {file_path}")
                            img = img.resize(max_size)
                            img.save(file_path)

                except (IOError, SyntaxError, ValueError, Image.DecompressionBombError) as e:
                    print(f"❌ Removing corrupt/oversized image: {file_path} due to error: {e}")
                    os.remove(file_path)  # Delete the corrupted/oversized file

# Run for both train and validation datasets
remove_or_resize_large_images("/content/drive/MyDrive/train")
remove_or_resize_large_images("/content/valid_0")

print("✅ Large images resized & corrupt images removed!")


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


⚠️ Resizing large image: /content/valid_0/HighSpatter/highgh_6.jpg
⚠️ Resizing large image: /content/valid_0/HighSpatter/highgh_14.jpg
✅ Large images resized & corrupt images removed!


In [ ]:
import tensorflow as tf
import os

# Define paths
train_dir = "/content/drive/MyDrive/train"
valid_dir = "/content/valid_0"

# Define image size and batch size
IMG_SIZE = (224, 224)  # Standard for ResNet
BATCH_SIZE = 32


In [ ]:

# Load datasets
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)


Found 2132 files belonging to 9 classes.
Found 8 files belonging to 9 classes.


In [ ]:

# Get class names
class_names = train_dataset.class_names
print(f"Class names: {class_names}")

# Normalize data
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))


Class names: ['Drop', 'Flow', 'Pattern', 'Pool', 'Swipe', 'Wipe', 'highspatter', 'low velocity blood spatter', 'mediumspatter']


In [ ]:

# Data Augmentation
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
])


In [ ]:

# Load pre-trained ResNet50V2
base_model = tf.keras.applications.ResNet50V2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights="imagenet"
)
base_model.trainable = False  # Freeze base layers initially


94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:

# Define model
inputs = tf.keras.Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = base_model(x, training=False)  # No batch norm update during freezing
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(len(class_names), activation="softmax")(x)

model = tf.keras.Model(inputs, outputs)


In [ ]:

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)


In [ ]:
import tensorflow as tf
import os

# Define paths
train_dir = "/content/drive/MyDrive/train"
valid_dir = "/content/valid_0"

# Verify that directories exist
if not os.path.exists(train_dir) or not os.path.exists(valid_dir):
    raise FileNotFoundError("❌ Training or Validation directory does not exist!")

# Define image size and batch size
IMG_SIZE = (224, 224)  # Standard for ResNet
BATCH_SIZE = 32

# Re-load datasets after removing missing files
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True
)

val_dataset = tf.keras.utils.image_dataset_from_directory(
    valid_dir,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42
)

# Get updated class names
class_names = train_dataset.class_names
print(f"✅ Updated Class Names: {class_names}")

# Normalize data
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
val_dataset = val_dataset.map(lambda x, y: (normalization_layer(x), y))

# Now try training again
EPOCHS = 10
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=EPOCHS
)

print("✅ Training started successfully!")


Found 2132 files belonging to 9 classes.
Found 8 files belonging to 9 classes.
✅ Updated Class Names: ['Drop', 'Flow', 'Pattern', 'Pool', 'Swipe', 'Wipe', 'highspatter', 'low velocity blood spatter', 'mediumspatter']
Epoch 1/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 423s 6s/step - accuracy: 0.2215 - loss: 2.6186 - val_accuracy: 0.2500 - val_loss: 2.2103
Epoch 2/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 404s 6s/step - accuracy: 0.5044 - loss: 1.4940 - val_accuracy: 0.1250 - val_loss: 2.4548
Epoch 3/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 450s 6s/step - accuracy: 0.6560 - loss: 1.0980 - val_accuracy: 0.1250 - val_loss: 2.7899
Epoch 4/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 432s 6s/step - accuracy: 0.7372 - loss: 0.8416 - val_accuracy: 0.1250 - val_loss: 3.2497
Epoch 5/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 441s 6s/step - accuracy: 0.7688 - loss: 0.7364 - val_accuracy: 0.1250 - val_loss: 3.6896
Epoch 6/10
67/67 ━━━━━━━━━━━━━━━━━━━━ 397s 6s/step - accuracy: 0.7879 - loss: 0.6545 - val_accuracy: 0.1250 - val_loss: 3.8766
Epoch 7/10
67/67 ━━━━

In [ ]:
model.save("/content/bloodstain_model.keras")
print("✅ Model saved successfully in .keras format!")

✅ Model saved successfully in .keras format!
